# 如何查看工具调用（函数式API）

!!!信息“先决条件”
本指南假设您熟悉以下内容：

- 使用[中断](../../concepts/ human_in_the_loop/#interrupt) 实施[人机循环](../../concepts/ human_in_the_loop) 工作流程
- [如何使用功能 API 创建 ReAct 代理](../../how-tos/react-agent-from-scratch-function)

本指南演示如何使用 LangGraph [Functional API](../../concepts/featured_api) 在 ReAct 代理中实现人机交互工作流程。

我们将基于[如何使用功能 API 创建 ReAct 代理](../../how-tos/react-agent-from-scratch-function) 指南中创建的代理进行构建。

具体来说，我们将演示如何在执行之前检查[聊天模型](https://js.langchain.com/docs/concepts/chat_models/)生成的[工具调用](https://js.langchain.com/docs/concepts/tool_calling/)。这可以通过在应用程序的关键点使用 [interrupt](../../concepts/ human_in_the_loop/#interrupt) 函数来完成。

**预览**：

我们将实现一个简单的函数，用于检查从聊天模型生成的工具调用，并从应用程序的 [entrypoint](../../concepts/function_api/#entrypoint) 内部调用它：
```ts
function reviewToolCall(toolCall: ToolCall): ToolCall | ToolMessage {
  // Interrupt for human review
  const humanReview = interrupt({
    question: "Is this correct?",
    tool_call: toolCall,
  });

  const { action, data } = humanReview;

  if (action === "continue") {
    return toolCall;
  } else if (action === "update") {
    return {
      ...toolCall,
      args: data,
    };
  } else if (action === "feedback") {
    return new ToolMessage({
      content: data,
      name: toolCall.name,
      tool_call_id: toolCall.id,
    });
  }
  throw new Error(`Unsupported review action: ${action}`);
}
```

## 设置

!!!注意兼容性

本指南需要 `@langchain/langgraph>=0.2.42`。

首先，安装本示例所需的依赖项：
```bash
npm install @langchain/langgraph @langchain/openai @langchain/core zod
```
接下来，我们需要为 OpenAI 设置 API 密钥（我们将使用的 LLM）：
```typescript
process.env.OPENAI_API_KEY = "YOUR_API_KEY";
```
!!!提示“为 LangGraph 开发设置 [LangSmith](https://smith.langchain.com)”

注册 LangSmith 以快速发现问题并提高 LangGraph 项目的性能。LangSmith 允许您使用跟踪数据来调试、测试和监控使用 LangGraph 构建的 LLM 应用程序 — 在[此处](https://docs.smith.langchain.com)了解有关如何开始的更多信息

## 定义模型和工具

让我们首先定义我们将用于示例的工具和模型。正如在 [ReAct 代理指南](../../how-tos/react-agent-from-scratch-function) 中一样，我们将使用一个占位符工具来获取某个位置的天气描述。

在本示例中，我们将使用 [OpenAI](https://js.langchain.com/docs/integrations/providers/openai/) 聊天模型，但任何[支持工具调用](https://js.langchain.com/docs/integrations/chat/) 的模型就足够了。

In [1]:
import { ChatOpenAI } from "@langchain/openai";
import { tool } from "@langchain/core/tools";
import { z } from "zod";

const model = new ChatOpenAI({
  model: "gpt-4o-mini",
});

const getWeather = tool(async ({ location }) => {
  // 这是实际实现的占位符
  const lowercaseLocation = location.toLowerCase();
  if (lowercaseLocation.includes("sf") || lowercaseLocation.includes("san francisco")) {
    return "It's sunny!";
  } else if (lowercaseLocation.includes("boston")) {
    return "It's rainy!";
  } else {
    return `I am not sure what the weather is in ${location}`;
  }
}, {
  name: "getWeather",
  schema: z.object({
    location: z.string().describe("Location to get the weather for"),
  }),
  description: "Call to get the weather from a specific location.",
});

const tools = [getWeather];

## 定义任务

我们的[任务](../../concepts/function_api/#task)与[ReAct代理指南](../../how-tos/react-agent-from-scratch-function)保持不变：

1. **调用模型**：我们想要使用消息列表查询我们的聊天模型。
2. **调用工具**：如果我们的模型生成工具调用，我们想要执行它们。

In [2]:
import {
  type BaseMessageLike,
  AIMessage,
  ToolMessage,
} from "@langchain/core/messages";
import { type ToolCall } from "@langchain/core/messages/tool";
import { task } from "@langchain/langgraph";

const toolsByName = Object.fromEntries(tools.map((tool) => [tool.name, tool]));

const callModel = task("callModel", async (messages: BaseMessageLike[]) => {
  const response = await model.bindTools(tools).invoke(messages);
  return response;
});

const callTool = task(
  "callTool",
  async (toolCall: ToolCall): Promise<AIMessage> => {
    const tool = toolsByName[toolCall.name];
    const observation = await tool.invoke(toolCall.args);
    return new ToolMessage({ content: observation, tool_call_id: toolCall.id });
    // 也可以直接将toolCall传递到工具中以返回ToolMessage
    // 返回 tool.invoke(toolCall);
  });

## 定义入口点

为了在执行前检查工具调用，我们添加了一个调用 [interrupt](../../concepts/ human_in_the_loop/#interrupt) 的 `reviewToolCalls` 函数。当调用此函数时，执行将暂停，直到我们发出命令来恢复它。

给定一个工具调用，我们的函数将 `interrupt` 进行人工审查。那时我们可以：

- 接受工具调用；
- 修改工具调用并继续；
- 生成自定义工具消息（例如，指示模型重新格式化其工具调用）。

我们将在下面的[使用示例](#usage)中演示这三种情况。

In [3]:
import { interrupt } from "@langchain/langgraph";

function reviewToolCall(toolCall: ToolCall): ToolCall | ToolMessage {
  // 中断人工审核
  const humanReview = interrupt({
    question: "Is this correct?",
    tool_call: toolCall,
  });

  const { action, data } = humanReview;

  if (action === "continue") {
    return toolCall;
  } else if (action === "update") {
    return {
      ...toolCall,
      args: data,
    };
  } else if (action === "feedback") {
    return new ToolMessage({
      content: data,
      name: toolCall.name,
      tool_call_id: toolCall.id,
    });
  }
  throw new Error(`Unsupported review action: ${action}`);
}

我们现在可以更新我们的 [entrypoint](../../concepts/tical_api/#entrypoint) 以查看生成的工具调用。如果工具调用被接受或修改，我们将以与以前相同的方式执行。否则，我们只需附加人类提供的 `ToolMessage`。

!!!提示

先前任务的结果（在本例中为初始模型调用）将被保留，以便它们不会在 `interrupt` 之后再次运行。

In [4]:
import {
  MemorySaver,
  addMessages,
  entrypoint,
  getPreviousState,
} from "@langchain/langgraph";

const checkpointer = new MemorySaver();

const agent = entrypoint({
  checkpointer,
  name: "agent",
}, async (messages: BaseMessageLike[]) => {
  const previous = getPreviousState<BaseMessageLike[]>() ?? [];
  let currentMessages = addMessages(previous, messages);
  let llmResponse = await callModel(currentMessages);
  while (true) {
    if (!llmResponse.tool_calls?.length) {
      break;
    }
    // 检查工具调用
    const toolResults: ToolMessage[] = [];
    const toolCalls: ToolCall[] = [];
    
    for (let i = 0; i < llmResponse.tool_calls.length; i++) {
      const review = await reviewToolCall(llmResponse.tool_calls[i]);
      if (review instanceof ToolMessage) {
        toolResults.push(review);
      } else { // is a validated tool call
        toolCalls.push(review);
        if (review !== llmResponse.tool_calls[i]) {
          llmResponse.tool_calls[i] = review;
        }
      }
    }
    // 执行剩余的工具调用
    const remainingToolResults = await Promise.all(
      toolCalls.map((toolCall) => callTool(toolCall))
    );
    
    // 附加到消息列表
    currentMessages = addMessages(
      currentMessages,
      [llmResponse, ...toolResults, ...remainingToolResults]
    );

    // 再次调用模型
    llmResponse = await callModel(currentMessages);
  }
  // 生成最终响应
  currentMessages = addMessages(currentMessages, llmResponse);
  return entrypoint.final({
    value: llmResponse,
    save: currentMessages
  });
});

### 用法

让我们演示一些场景。

In [5]:
import { BaseMessage, isAIMessage } from "@langchain/core/messages";

const prettyPrintMessage = (message: BaseMessage) => {
  console.log("=".repeat(30), `${message.getType()} message`, "=".repeat(30));
  console.log(message.content);
  if (isAIMessage(message) && message.tool_calls?.length) {
    console.log(JSON.stringify(message.tool_calls, null, 2));
  }
}

const printStep = (step: Record<string, any>) => {
  if (step.__metadata__?.cached) {
    return;
  }
  for (const [taskName, result] of Object.entries(step)) {
    if (taskName === "agent") {
      continue; // just stream from tasks
    }
    
    console.log(`\n${taskName}:`);
    if (taskName === "__interrupt__" || taskName === "reviewToolCall") {
      console.log(JSON.stringify(result, null, 2));
    } else {
      prettyPrintMessage(result);
    }
  }
};

### 接受工具调用

要接受工具调用，我们只需在 `Command` 中提供的数据中指示工具调用应该通过。

In [6]:
const config = {
  configurable: {
    thread_id: "1"
  }
};

const userMessage = {
  role: "user",
  content: "What's the weather in san francisco?"
};
console.log(userMessage);

const stream = await agent.stream([userMessage], config);

for await (const step of stream) {
  printStep(step);
}

{ role: 'user', content: "What's the weather in san francisco?" }



callModel:
============================== ai message ==============================

[
  {
    "name": "getWeather",
    "args": {
      "location": "San Francisco"
    },
    "type": "tool_call",
    "id": "call_pe7ee3A4lOO4Llr2NcfRukyp"
  }
]

__interrupt__:
[
  {
    "value": {
      "question": "Is this correct?",
      "tool_call": {
        "name": "getWeather",
        "args": {
          "location": "San Francisco"
        },
        "type": "tool_call",
        "id": "call_pe7ee3A4lOO4Llr2NcfRukyp"
      }
    },
    "when": "during",
    "resumable": true,
    "ns": [
      "agent:dcee519a-80f5-5950-9e1c-e8bb85ed436f"
    ]
  }
]


In [7]:
import { Command } from "@langchain/langgraph";

// 突出显示下一行
const humanInput = new Command({
  // 突出显示下一行
  resume: {
    // 突出显示下一行
    action: "continue",
    // 突出显示下一行
  },
  // 突出显示下一行
});

const resumedStream = await agent.stream(humanInput, config)

for await (const step of resumedStream) {
  printStep(step);
}


callTool:
============================== tool message ==============================
It's sunny!

callModel:
============================== ai message ==============================
The weather in San Francisco is sunny!


### 修改工具调用

要修改工具调用，我们可以提供更新的参数。

In [8]:
const config2 = {
  configurable: {
    thread_id: "2"
  }
};

const userMessage2 = {
  role: "user",
  content: "What's the weather in san francisco?"
};

console.log(userMessage2);

const stream2 = await agent.stream([userMessage2], config2);

for await (const step of stream2) {
  printStep(step);
}

{ role: 'user', content: "What's the weather in san francisco?" }

callModel:
============================== ai message ==============================

[
  {
    "name": "getWeather",
    "args": {
      "location": "San Francisco"
    },
    "type": "tool_call",
    "id": "call_JEOqaUEvYJ4pzMtVyCQa6H2H"
  }
]

__interrupt__:
[
  {
    "value": {
      "question": "Is this correct?",
      "tool_call": {
        "name": "getWeather",
        "args": {
          "location": "San Francisco"
        },
        "type": "tool_call",
        "id": "call_JEOqaUEvYJ4pzMtVyCQa6H2H"
      }
    },
    "when": "during",
    "resumable": true,
    "ns": [
      "agent:d5c54c67-483a-589a-a1e7-2a8465b3ef13"
    ]
  }
]


In [9]:
// 突出显示下一行
const humanInput2 = new Command({
  // 突出显示下一行
  resume: {
    // 突出显示下一行
    action: "update",
    // 突出显示下一行
    data: { location: "SF, CA" },
    // 突出显示下一行
  },
  // 突出显示下一行
});

const resumedStream2 = await agent.stream(humanInput2, config2)

for await (const step of resumedStream2) {
  printStep(step);
}


callTool:
============================== tool message ==============================
It's sunny!

callModel:
============================== ai message ==============================
The weather in San Francisco is sunny!


这次运行的 LangSmith 跟踪信息特别丰富：

- 在[中断之前](https://smith.langchain.com/public/abf80a16-3e15-484b-bbbb-23017593bd39/r)中，我们生成位置`"San Francisco"`的工具调用。
- 在[恢复后](https://smith.langchain.com/public/233a7e32-a43e-4939-9c04-96fd4254ce65/r)的跟踪中，我们看到消息中的工具调用已更新为`"SF, CA"`。

### 生成自定义 ToolMessage

为了生成自定义 `ToolMessage`，我们提供消息的内容。在这种情况下，我们将要求模型重新格式化其工具调用。

In [10]:
const config3 = {
  configurable: {
    thread_id: "3"
  }
};

const userMessage3 = {
  role: "user",
  content: "What's the weather in san francisco?"
};

console.log(userMessage3);

const stream3 = await agent.stream([userMessage3], config3);

for await (const step of stream3) {
  printStep(step);
}

{ role: 'user', content: "What's the weather in san francisco?" }

callModel:
============================== ai message ==============================

[
  {
    "name": "getWeather",
    "args": {
      "location": "San Francisco"
    },
    "type": "tool_call",
    "id": "call_HNRjJLJo4U78dtk0uJ9YZF6V"
  }
]

__interrupt__:
[
  {
    "value": {
      "question": "Is this correct?",
      "tool_call": {
        "name": "getWeather",
        "args": {
          "location": "San Francisco"
        },
        "type": "tool_call",
        "id": "call_HNRjJLJo4U78dtk0uJ9YZF6V"
      }
    },
    "when": "during",
    "resumable": true,
    "ns": [
      "agent:6f313de8-c19e-5c3e-bdff-f90cdd68d0de"
    ]
  }
]


In [11]:
// 突出显示下一行
const humanInput3 = new Command({
  // 突出显示下一行
  resume: {
    // 突出显示下一行
    action: "feedback",
    // 突出显示下一行
    data: "Please format as <City>, <State>.",
    // 突出显示下一行
  },
  // 突出显示下一行
});

const resumedStream3 = await agent.stream(humanInput3, config3)

for await (const step of resumedStream3) {
  printStep(step);
}


callModel:
============================== ai message ==============================

[
  {
    "name": "getWeather",
    "args": {
      "location": "San Francisco, CA"
    },
    "type": "tool_call",
    "id": "call_5V4Oj4JV2DVfeteM4Aaf2ieD"
  }
]

__interrupt__:
[
  {
    "value": {
      "question": "Is this correct?",
      "tool_call": {
        "name": "getWeather",
        "args": {
          "location": "San Francisco, CA"
        },
        "type": "tool_call",
        "id": "call_5V4Oj4JV2DVfeteM4Aaf2ieD"
      }
    },
    "when": "during",
    "resumable": true,
    "ns": [
      "agent:6f313de8-c19e-5c3e-bdff-f90cdd68d0de"
    ]
  }
]


重新格式化后，我们就可以接受它：

In [12]:
// 突出显示下一行
const continueCommand = new Command({
  // 突出显示下一行
  resume: {
    // 突出显示下一行
    action: "continue",
    // 突出显示下一行
  },
  // 突出显示下一行
});

const continueStream = await agent.stream(continueCommand, config3)

for await (const step of continueStream) {
  printStep(step);
}


callTool:
============================== tool message ==============================
It's sunny!

callModel:
============================== ai message ==============================
The weather in San Francisco, CA is sunny!
